## River Crossing – Solution

We now define the `Node` class for the missionaries and cannibals problem based on the "infrastructure for search algorithms" shown in Section 3.3.1 of R&N (2016). Note that the arguments `parent_node` and `action` are **not** needed to implement the breadth-first-search if the only goal is to reach the goal state. However, if we want to compute one actual path from start to goal (that is, without redundant nodes that were explored during search), we can write a function that recursively checks for the parent node of the goal node. Such a function is shown at the end of the notebook.

In [1]:
import numpy as np

In [2]:
class Node:
    def __init__(self, m_wrong_side, c_wrong_side, boat_wrong_side, parent_node):
        self.m_wrong_side = m_wrong_side
        self.c_wrong_side = c_wrong_side
        self.boat_wrong_side = boat_wrong_side
        self.parent_node = parent_node
        
        # Storing all three position arguments again in an array is sort of redundant but will
        # simplify a lot of the following code, especially the addition and substraction of "actions".
        self.state = np.array([self.m_wrong_side, self.c_wrong_side, self.boat_wrong_side])

    def is_goal_state(self):
        # Returns a boolean. True, if all m, c, and the boat are on the right side.
        return np.all(self.state == 0)

    def get_child_node(self, action):
        if self.boat_wrong_side == 1:
            new_state = self.state - action
        elif self.boat_wrong_side == 0:
            new_state = self.state + action
        else:
            raise ValueError('"boat_wrong_side" has to be either 0 or 1.')
        child_node = Node(new_state[0], new_state[1], new_state[2], parent_node=self)
        return child_node

    def is_valid(self):
        if 0 <= self.m_wrong_side <= 3 \
            and 0 <= self.c_wrong_side <= 3 \
            and (self.m_wrong_side == 0 or self.c_wrong_side <= self.m_wrong_side) \
            and (self.m_wrong_side == 3 or self.c_wrong_side >= self.m_wrong_side):
                return True
        else:
            return False
        
    def __eq__(self, other):
        # Define how to check equality between Node objects. 
        # This is useful to check, for example, whether an object is in a set or not.
        return np.array_equal(self.state, other.state)
    
    def __hash__(self):
        # User-defined objects are hashable by default. However, when defining an __eq__() method,
        # one needs to define a new hash function as well. See, for example,
        # https://stackoverflow.com/a/29434112
        # We can just hash the state. Note that the value that we are trying to hash (self.state)
        # is mutable because it is a np.array. Hash values need to be immutable; we thus transform
        # the np.array into a tuple before hashing.
        return hash(tuple(self.state))

    def __str__(self):
        # Define how the node is printed
        return ("Node <" + str(self.m_wrong_side) + "," +
                           str(self.c_wrong_side) + "," +
                           str(self.boat_wrong_side) + ">")
    

The `Game` class implements the actual breadth-first search and stores the possible actions as well as the initial node.

In [3]:
class Game:
    def __init__(self):
        self.initial_node = Node(m_wrong_side=3, c_wrong_side=3, boat_wrong_side=1, 
                                 parent_node="initial")
        self.actions = [np.array([1, 0, 1]),
                        np.array([2, 0, 1]),
                        np.array([0, 1, 1]),
                        np.array([0, 2, 1]),
                        np.array([1, 1, 1])]

    def breadth_first_search(self):
        if self.initial_node.is_goal_state():
            return self.initial_node

        frontier = []
        frontier.append(self.initial_node)
        explored = set()

        while True:
            if len(frontier) == 0:
                return "Failure"
            # FIFO: get first element
            node = frontier.pop(0)
            explored.add(node)
            print("Exploring", node, "...")
            for action in self.actions:  
                child = node.get_child_node(action)
                if child.is_valid() and (child not in explored) and (child not in frontier):
                    if child.is_goal_state():
                        return child
                    frontier.append(child)

In [4]:
g = Game()
goal_state = g.breadth_first_search()
print("Found goal state", goal_state)

Exploring Node <3,3,1> ...
Exploring Node <3,2,0> ...
Exploring Node <3,1,0> ...
Exploring Node <2,2,0> ...
Exploring Node <3,2,1> ...
Exploring Node <3,0,0> ...
Exploring Node <3,1,1> ...
Exploring Node <1,1,0> ...
Exploring Node <2,2,1> ...
Exploring Node <0,2,0> ...
Exploring Node <0,3,1> ...
Exploring Node <0,1,0> ...
Exploring Node <1,1,1> ...
Found goal state Node <0,0,0>


This implementation of BFS could be made more efficient, for example, by _not_ exploring the action that was applied to the parent node to generate the node. The current algorithm unnecessarily computes the parent node of each node although this one is already known and in the `explored` set!

### Extracting the path to the goal state

Note that the breadth-first search executed above explored nodes that were "dead ends" and are not actually part of the path to the goal node. We can extract this path by recursively accessing the parent node, starting with the goal node.

In [5]:
def find_path_to_start(goal_node):
    # Recursively check parent node(s) of the goal_node
    def get_parent_node(node):
        parent_node = node.parent_node
        print("-->", parent_node)
        if isinstance(parent_node.parent_node, str) and parent_node.parent_node == "initial":
            return parent_node
        else:
            get_parent_node(parent_node) 
    
    print("Goal node:", goal_node)
    start_node = get_parent_node(goal_node)
    return start_node
    
start_node = find_path_to_start(goal_state)   

Goal node: Node <0,0,0>
--> Node <1,1,1>
--> Node <0,1,0>
--> Node <0,3,1>
--> Node <0,2,0>
--> Node <2,2,1>
--> Node <1,1,0>
--> Node <3,1,1>
--> Node <3,0,0>
--> Node <3,2,1>
--> Node <3,1,0>
--> Node <3,3,1>


$<3,3,1>$ is indeed the start node. We found a path of depth 11 using breadth-first search. 